> <b>参考自：[all-agentic-architectures](https://github.com/FareedKhan-dev/all-agentic-architectures/tree/main)</b>

# Tool Use (工具调用架构)

## 1. 定义 (Definition)
**Tool Use** 是一种赋予大语言模型（LLM）驱动的智能体（Agent）调用外部函数或 API 能力的架构。
* **自主性**：智能体会自主判断用户问题是否超出了其内部知识范围。
* **决策性**：智能体能够决定在何时、调用哪一个特定的工具来获取必要的信息。

---

## 2. 宏观工作流 (Workflow)
工具调用的过程遵循“接收-决策-执行-观察-综合”的循环：

1.  **接收请求 (Receive Query)**：智能体接收来自用户的原始指令。
2.  **决策阶段 (Decision)**：分析查询内容及可用工具列表，判断是否需要外部工具协助。
3.  **执行动作 (Action)**：若需要工具，智能体按格式要求生成调用指令（如特定函数名及参数）。
4.  **获取观察 (Observation)**：系统执行该调用，并将结果（即“观察到的数据”）反馈给智能体。
5.  **综合输出 (Synthesis)**：智能体将工具返回的信息整合进推理过程中，生成最终的、有据可依的回答。



---

## 3. 应用场景 (Use Cases)
* **研究助手**：通过调用网络搜索 API，回答需要**即时信息**的问题。
* **企业级助手**：通过查询公司内部数据库，回答如“上周新增用户数”等**业务数据**问题。
* **科学与数学任务**：利用计算器或 WolframAlpha 等**计算引擎**，处理模型自身不擅长的精确计算。

---

## 4. 优缺点分析 (Pros & Cons)

### ✅ 优点
* **事实增强 (Factual Grounding)**：通过获取实时数据，显著减少模型的“幻觉”现象。
* **可扩展性 (Extensibility)**：只需添加新工具，即可不断扩展智能体的能力边界。

### ❌ 缺点
* **集成成本 (Integration Overhead)**：需要精细的“管道”工程，包括定义接口、处理 API 密钥以及管理工具失效的情况。
* **工具信任 (Tool Trust)**：回答的质量高度依赖于工具的准确性；如果工具提供错误信息，智能体也难以产出正确答案。

# 0、基础和配置

## 0.1 安装核心依赖
在构建基于工具调用的智能体前，我们需要安装一套标准库。这些库分别负责任务编排、环境变量管理、结果展示以及外部工具接入。

### 🛠️ 需要安装的库及其作用
| 库名称 | 作用描述 |
| :--- | :--- |
| **`langchain-openai`** | 用于与 OpenAI 的模型（如 GPT-4）进行交互的 LangChain 集成库。 |
| **`langgraph`** | 用于构建有状态的、多参与者（Multi-agent）应用的库，支持创建复杂的循环工作流。 |
| **`python-dotenv`** | 用于从 `.env` 文件中读取 API 密钥等环境变量，保护敏感信息。 |
| **`rich`** | 一个 Python 库，用于在终端输出格式漂亮、带颜色的文本和表格，方便调试。 |
| **`tavily-python`** | **核心搜索工具**。Tavily 是专为 AI 智能体设计的搜索引擎，能够返回结构化的搜索结果。 |

### 💻 安装命令
你可以通过以下 pip 命令一键安装所有依赖：

```bash
pip install -U langchain-openai langchain langgraph python-dotenv rich tavily-python langchain_community pydantic

In [3]:
%pip install -U langchain-openai langchain langgraph python-dotenv rich tavily-python langchain_community pydantic

  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.0 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 821.2 kB/s eta 0:00:00a 0:00:01
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Users/okonma/Desktop/PyProj/AgentArchitechtures/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 0.2 导入库并设置密钥

在这一步中，我们将导入核心模块，并使用 `python-dotenv` 安全地加载 API 密钥。

### 🔑 必需的 API 密钥
为了让智能体正常工作，你需要准备以下三类 Key：
1.  **OpenAI Key**：驱动大语言模型（LLM）进行推理和决策。
2.  **LangSmith Key**：用于**追踪（Tracing）**。它可以记录智能体的每一步运行轨迹，是调试和优化 Agent 复杂逻辑的必备工具。
3.  **Tavily Key**：赋予智能体联网搜索的能力。

In [1]:
import os
import json
from  typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# LangChain组件
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, ToolMessage
from pydantic import BaseModel, Field

# LangGraph组件
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# 输出美化
from rich.console import Console
from rich.markdown import Markdown

# API密钥
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Tool Use"

for key in ["OPENAI_API_KEY", "LANGCHAIN_API_KEY", "DASHSCOPE_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} not found.")

print("Keys loaded successfully.")

Keys loaded successfully.


# 1、创建 Agent 工具

## 1.1 创建并测试 Web Search 工具

在这一阶段，我们将实例化 `TavilySearchResults` 工具。这是智能体与外部世界连接的第一个通道。

### 🎯 核心概念：描述的重要性
定义工具时，最关键的部分是它的**描述（Description）**。
* **决策依据**：LLM 并不通过底层代码了解工具，而是阅读这段**自然语言描述**。
* **精准引导**：一个清晰、准确的描述能告诉 LLM 该工具的功能、适用场景以及参数含义。如果描述模糊，Agent 可能会在不该调用时误用，或在需要时忽略它。

In [3]:
# 初始化工具，可以设置最多的结果数量以保证上下文的精简
search_tool = TavilySearchResults(max_results=2)

# 给工具定义清楚的工具名和描述是很重要的
search_tool.name = "web_search"
search_tool.description = "A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs."

tools=[search_tool]
print(f"Tool '{search_tool.name}' created with description: '{search_tool.description}'")

console = Console()

# 直接测试工具并观察它的输出格式
print("\n--- Testing the tool directly ---")
test_query = "常熟阿诺头为什么尖尖的?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query:[/bold green] {test_query}")
console.print("\n[bold green]Result:[/bold green]")
console.print(test_result)

Tool 'web_search' created with description: 'A tool that can be used to search the internet for up-to-date information on any topic, including news, events, and current affairs.'

--- Testing the tool directly ---


Query: 常熟阿诺头为什么尖尖的?

Result:

[
    {
        'title': '全网烂梗最多的健身网红，终于把自己练成了人类最抽象的样子',
        'url': 'https://news.qq.com/rain/a/20240815A09QQ700',
        'content': 
'所以，只有以下两种可能性。要么诺神是自宇宙开天辟地以来，抗药能力最强的碳基生物，是比太上老君更给劲的人体炼丹炉；要
么诺神买到了假药还天天吃，不能说没有一点效果，但作用在脑壳上的效果，比肌肉更明显。\n\n据说群勃龙会对大脑产生各种未
知后果，导致类如阿兹海默症等精神疾病，又或者诺神本就在语言方面天赋异禀，逻辑远超常人。时间久了，常熟阿诺的各种语录
也开始在网上流传，俗称“诺言诺语”。\n\n比如，当阿诺被问到头顶怎么尖尖的，诺神反手回问了一百个“那我问你”。一顿下来嘴
巴先力竭了，今天的训练量又达标了。\n\n完整版如下： [...] > 
那我问你，你是男的女的，如果你是女的你说这样的话啊，那我问你，你你你是女孩子那我问你，那那你那你头顶是不是是不是尖
的，那我那我问你，你头顶是尖的呢，还是秃顶的啊，还是染黄色染红色的，那我问你，啊，还是戴假发的，如果是你如果你是男
的，那我问你啊，你说我的头是尖的，那我问你，那你是不是秃头，那你是不是光头啊，你是光头还是有头发的啊，那我问你，我
头顶，我头尖怎么了，我就尖怎么了，哎我头尖怎么了，我就头顶尖怎么了，我就头顶尖你难道你看不惯吗，啊我头顶就是尖的怎
么了我就是尖，我就是要尖怎么了啊，你看不惯吗，啊那我问你......\n\n他对一场比赛是否在常熟举办，也发表了自己的独到见
解，爱因斯坦听见都得修改相对论。不过说句公道话，诺神这里只是不小心省略了主语，他实际想表达的意思应该是“传言常熟可能
举办DMS，但是他看来常熟举办DMS不太可能”。\n\n当观众问他健身用什么补剂，诺神表示自己连蛋白粉都不用的，最多也就用点蛋
白粉。\n\n谈到相亲相爱的师傅李亚强，诺神更是表示两人的关系血浓于水，“师父对待我就像父亲一样看待”。 [...] 
不同于丁真、范马家族等草根抽象人物，常熟阿诺的家境其实相当富裕，属于官三代+富三代的王炸开局。按照寻常发展轨迹，阿诺
本该会完全凭借自己的努力奋斗，考进常熟当地的某国家单位，完全没必要转行健美，走一条最艰难的路。\n\n但网上一些关于阿
诺的过往轶事，或许揭露了背后的原因。据称，诺神打上小学起，就展现出了惊人的学习天赋，属于每门课考试都爆砍个位数的那
种，发展十分均衡。这也让某些不合理变成了合理。\n\n在毕业之后，诺神还是在家人的安排下，和李老八一样，成为了一名光荣
的辅警。然而据说他却在扫黄打非行动中，暗中给小姐姐们通风报信。也因其过于超前的意识形态，最终连辅警这条路也走不通了
。\n\n所以，不管因为个人的奋斗，还是历史的进程，常熟阿诺可能天生就该吃健美这碗饭。此后诺神化悲愤为力量，潜心专研健
身，并凭借稀世罕见的天赋和意志力，率先为健美圈带来了变革。\n\n常熟阿诺的成名之战，是他瞒着第一任师傅钱升，参加上海
健美比赛的事情。给可能不了解的朋友科普一下，健美运动员在参加比赛前，需要通过长时间的严苛训练，将体脂率降低到只有个
位数，并让身体大量脱水，从而凸显出肌肉线条，也就是俗称的“备赛”。',
        'score': 0.8215175
    },
    {
        'title': '네이버 중국어사전',
        'url': 'https://zh.dict.naver.com/',
        'content': '본문 영역으로 바로가기\n\n### HSK 단어\n\nHSK 고등단어\n\n把关\n[ bǎ//guān ]\n\n 관문(關門)을 
지키다. {전용} 책임을 지다.\n\n4급 단어\n\n维护\n[ wéihù ]\n\n 지키다. 유지하고 보호하다. 옹호[수호]하다.\n\n### 
주제별 찾기\n\n #HSK 등급 단어\n #지역별 악센트 - 핵심 단어\n #지역별 악센트 - 생활 문장\n #방언으로 배우는 
중국어\n\n### 이용자 참여사전 - 오픈사전PRO\n\n누구나 쉽게 만들수 있는 오픈사전PRO와 함께 지금 바로 나만의 사전을 
만들어 보세요!\n\n사전 만들기\n\n 단어장\n 작은창 사전\n 오픈사전\n\n### 공지사항 새로운 공시자항 등록됨\n\n### HSK
등급별 단어 찾기\n\n 1급\n 2급\n 3급\n 4급\n 5급\n 6급\n 고등\n\n### 지역별 악센트 - 핵심 단어\n\n### 지역별 악센트
- 생활 문장\n\n### 방언으로 배우는 중국어\n\n 광둥\n 둥베이\n 민난\n 상하이\n 쓰촨\n 안후이\n 웨이하이\n 지난\n 
허난\n 후난\n\n### 사전 사용팁 [...] 그려서 찾는 필기인식기\n  모르는 중국어는   \n   직접 그려서 찾을 수 
있습니다.\n 편리하게 입력하기\n  입력기, 자동완성 등 다양한 기능으로   \n   편리하게 사전을 이용해 보세요.\n 독해를
도와주는 문장분석기\n  단어 단위로 뜻풀이 해주어   \n   문장을 쉽게 이해할 수 있습니다.\n\n이용약관 동의\n\n단어장 
이용약관에 동의하신 후 이용이 가능합니다.\n\n단어가 저장되었습니다.',
        'score': 0.054243922
    }
]

### 📝 输出结果讨论 (Discussion of the Output)

通过对 `web_search` 工具的直接测试，我们可以观察到工具返回的**原始数据结构**。

#### 1. 数据结构分析
测试显示，该工具返回的是一个**字典列表 (List of Dictionaries)**：
* 每个字典代表一个搜索结果。
* 包含核心字段：`url`（来源链接）和 `content`（网页内容摘要）。